In [1]:
%load_ext autoreload
%autoreload 2

In [24]:
import json
EXPERIENCE_PATH = './trainingdata/hansolo/young-han.json'
PROTECTED_PATH = './trainingdata/hansolo/young-han-protective.json'

new_experience = []
new_protected = []

# loop thru the original jsons
# for each object present, do the following:
# 1. create an empty dict
# 2. add the key "text"

# with open(EXPERIENCE_PATH, 'r') as f:
#     exp = json.load(f)
#     for obj in exp:
#         #print(obj)
#         temp = {'text': []}
#         temp['text'].append('<instruction>')
#         temp['text'].append(obj['instruction'])
#         temp['text'].append('<input>')
#         temp['text'].append(obj['input'])
#         temp['text'].append('<output>')
#         temp['text'].append(obj['output'])
#         temp['text'] = ' '.join(temp['text'])
#         new_experience.append(temp)
#     f.close()
# print(len(new_experience))
# # create a json file with the new experience
# with open('./trainingdata/hansolo/young-han-new.jsonl', 'w') as f:
#     for entry in new_experience:
#         json.dump(entry, f)
#         f.write('\n')
#     # json.dump(new_experience, f)
#     f.close()

with open(PROTECTED_PATH, 'r') as f:
    exp = json.load(f)
    for obj in exp:
        #print(obj)
        temp = {'text': []}
        temp['text'].append('<instruction>')
        temp['text'].append(obj['instruction'])
        temp['text'].append('<input>')
        temp['text'].append(obj['input'])
        temp['text'].append('<output>')
        temp['text'].append(obj['output'])
        temp['text'] = ' '.join(temp['text'])
        new_protected.append(temp)
    f.close()

with open('./trainingdata/hansolo/young-han-protective-new.jsonl', 'w') as f:
    for entry in new_protected:
        json.dump(entry, f)
        f.write('\n')
    # json.dump(new_experience, f)
    f.close()


In [37]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import json
import together
from apikey import KEY

EXPERIENCE_PATH = './trainingdata/hansolo/young-han-new.jsonl'
# PROTECTED_PATH = './trainingdata/hansolo/young-han-protective-new.jsonl'

together.api_key = KEY

model_list = together.Models.list()

print(f"{len(model_list)} models available")

# print the first 10 models on the menu
# model_names = [model_dict['name'] for model_dict in model_list]
# print(model_names[:10])

103 models available


In [38]:
resp = together.Files.upload(file=EXPERIENCE_PATH)
file_id = resp["id"]

Uploading ./trainingdata/hansolo/young-han-new.jsonl: 100%|██████████| 781k/781k [00:00<00:00, 1.92MB/s]


In [39]:
files_list = together.Files.list()
files_list['data']

[{'filename': 'young-han-new.jsonl',
  'bytes': 799730,
  'created_at': 1699945068,
  'id': 'file-bb280f38-2b64-42a9-9349-6e7eb2ad2d05',
  'purpose': 'fine-tune',
  'object': 'file',
  'LineCount': 0,
  'Processed': True}]

In [40]:
resp = together.Finetune.create(
  training_file = 'file-bb280f38-2b64-42a9-9349-6e7eb2ad2d05',
  model = 'togethercomputer/llama-2-70b-chat',
  n_epochs = 3,
  n_checkpoints = 1,
  batch_size = 4,
  learning_rate = 1e-5,
  suffix = 'my-demo-finetune',
  wandb_api_key = KEY,
)

fine_tune_id = resp['id']
print(resp)

{'training_file': 'file-bb280f38-2b64-42a9-9349-6e7eb2ad2d05', 'validation_file': '', 'model_output_name': 'mifu67@stanford.edu/llama-2-70b-chat-my-demo-finetune-2023-11-14-07-01-04', 'model_output_path': 's3://together-dev/finetune/6552b4a556bb2d3952ed7a14/mifu67@stanford.edu/llama-2-70b-chat-my-demo-finetune-2023-11-14-07-01-04/ft-3a9730a4-4fd6-4330-bb72-8327eede69b4', 'Suffix': 'my-demo-finetune', 'model': 'togethercomputer/llama-2-70b-chat', 'n_epochs': 3, 'n_checkpoints': 1, 'batch_size': 32, 'learning_rate': 1e-05, 'user_id': '6552b4a556bb2d3952ed7a14', 'lora': False, 'lora_r': 8, 'lora_alpha': 8, 'lora_dropout': 0, 'staring_epoch': 0, 'training_offset': 0, 'checkspoint_path': '', 'random_seed': '', 'created_at': '2023-11-14T07:01:04.613Z', 'updated_at': '2023-11-14T07:01:04.613Z', 'status': 'pending', 'owner_address': '0xebd597774b313d8e8d22af4629b75ec7d567f678', 'id': 'ft-3a9730a4-4fd6-4330-bb72-8327eede69b4', 'job_id': '', 'token_count': 0, 'param_count': 0, 'total_price': 0, 

In [50]:
resp = together.Finetune.list()
# resp['data'][0]
print(len(resp['data']))

3


In [49]:
together.Finetune.cancel('ft-3a9730a4-4fd6-4330-bb72-8327eede69b4')

{'training_file': 'file-bb280f38-2b64-42a9-9349-6e7eb2ad2d05',
 'validation_file': '',
 'model_output_name': 'mifu67@stanford.edu/llama-2-70b-chat-my-demo-finetune-2023-11-14-07-01-04',
 'model_output_path': 's3://together-dev/finetune/6552b4a556bb2d3952ed7a14/mifu67@stanford.edu/llama-2-70b-chat-my-demo-finetune-2023-11-14-07-01-04/ft-3a9730a4-4fd6-4330-bb72-8327eede69b4',
 'Suffix': 'my-demo-finetune',
 'model': 'togethercomputer/llama-2-70b-chat',
 'n_epochs': 3,
 'n_checkpoints': 1,
 'batch_size': 32,
 'learning_rate': 1e-05,
 'user_id': '6552b4a556bb2d3952ed7a14',
 'lora': False,
 'lora_r': 8,
 'lora_alpha': 8,
 'lora_dropout': 0,
 'staring_epoch': 0,
 'training_offset': 0,
 'checkspoint_path': '',
 'random_seed': '',
 'created_at': '2023-11-14T07:01:04.613Z',
 'updated_at': '2023-11-14T07:11:27.86Z',
 'status': 'cancel_requested',
 'owner_address': '0xebd597774b313d8e8d22af4629b75ec7d567f678',
 'id': 'ft-3a9730a4-4fd6-4330-bb72-8327eede69b4',
 'job_id': '',
 'token_count': 0,
 'p